In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

In [4]:
data=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
test_ids=test["PassengerId"]

def clean(data):
    data=data.drop(["Ticket","Cabin","Name","PassengerId"],axis=1)
    cols=["SibSp","Parch","Fare","Age"]
    for col in cols:
        data[col].fillna(data[col].median(),inplace=True)
    data.Embarked.fillna("U",inplace=True)
    return data
data=clean(data)
test=clean(test)

# data.head(5)
test.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [5]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()

cols=["Sex","Embarked"]

for col in cols:
    data[col]=le.fit_transform(data[col])
    test[col]=le.transform(test[col])
    print(le.classes_)
data.head(5)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y=data["Survived"]
X=data.drop("Survived",axis=1)

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
X_train.shape

(712, 7)

In [150]:
# clf=LogisticRegression(random_state=0,max_iter=1000).fit(X_train,y_train)
# loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
# model=keras.Sequential([
#     keras.layers.BatchNormalization(),
#     keras.layers.Dense(25 , activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.01)),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dense(10,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.01)),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dense(1 , activation="linear")
# ])

# model.compile(optimizer='adam', loss=loss_fn,metrics=['accuracy'])
# model.fit(X_train,y_train,epochs=1000)

param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.5, 0.7, 1],
    'n_estimators':[10 ,50 ,100]
}

# model = xgb.XGBClassifier(n_estimators=10, max_depth=5, learning_rate=0.1, subsample=0.7, objective='binary:logistic')
xgb_model = xgb.XGBClassifier()

grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

print("Best set of hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# model.fit(X_train, y_train)
# predictions = xgb_model.predict(X_val)
# accuracy = accuracy_score(y_val, predictions)
# accuracy

final_model=grid_search.best_estimator_
final_model

Best set of hyperparameters:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.5}
Best score:  0.8342361863488623


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [159]:
final_model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [138]:
# predictions=clf.predict(X_val)
# from sklearn.metrics import accuracy_score
# accuracy_score(y_val,predictions)
y_predicted=model.predict(X_val)
sm_predicted = tf.nn.sigmoid(y_predicted).numpy()
for i in range(len(sm_predicted)):
    if(sm_predicted[i][0]>=0.5):
        sm_predicted[i][0]=1
    else:
        sm_predicted[i][0]=0

InvalidArgumentError: Value for attr 'T' of int32 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Sigmoid}}; Op<name=Sigmoid; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Sigmoid] name: 

In [156]:
submission_preds=final_model.predict(test)
submission_preds.shape
# sm_submission=tf.nn.sigmoid(submission_preds).numpy()

# for i in range(len(sm_submission)):
#     if(sm_submission[i][0]>=0.5):
#         sm_submission[i][0]=1
#     else:
#         sm_submission[i][0]=0
# result=sm_submission.flatten()
# result=result.astype(int)
# result

(418,)

In [157]:
df=pd.DataFrame({"PassengerId":test_ids.values,"Survived":submission_preds,})

In [158]:
df.to_csv("submission.csv",index=False)